# This is a sample Jupyter Notebook

Below is an example of a code cell. 
Put your cursor into the cell and press Shift+Enter to execute it and select the next one, or click 'Run Cell' button.

Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.

To learn more about Jupyter Notebooks in PyCharm, see [help](https://www.jetbrains.com/help/pycharm/ipython-notebook-support.html).
For an overview of PyCharm, go to Help -> Learn IDE features or refer to [our documentation](https://www.jetbrains.com/help/pycharm/getting-started.html).

https://raw.githubusercontent.com/la-rockoteque/Vestigium/refs/heads/main/out/Velum%20Cineris%3B%20Vestigium%20-%20Guide%20to%20Concord%20City.json


In [40]:
from diffusers import StableDiffusionPipeline
import torch

# pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4")
# pipe.to("mps")
# _ = pipe(prompt, num_inference_steps=1)
def generate_icon(category: str, name: str, image_size=(128, 128), background_color="white", text_color="black", font_size=24):
  return f"https://raw.githubusercontent.com/la-rockoteque/Vestigium/refs/heads/main/images/{category}/{inflection.underscore(name.replace('/', '_'))}.png"
  import os.path
  file_name = f"images/{category}/{inflection.underscore(name.replace('/', '_'))}.png"
  if os.path.isfile(file_name):
    return file_name
  prompt = f"{category} icon for a {category} named {name}"
  image = pipe(prompt).images[0]
  image.save(file_name)
  return file_name

# Spells

In [41]:
# %% [markdown]
# # Export Excel Spells to FoundryVTT Compendium JSON
#
# This notebook reads the "Spells" sheet from the Excel file and converts each row into a JSON object formatted for FoundryVTT.
# You may need to adjust the column names if they differ from your Excel file.

# %%
import pandas as pd
import json
from PIL import Image, ImageDraw, ImageFont
import inflection
import time

spells_url = "https://docs.google.com/spreadsheets/d/1I4FHncl40_xx1Udc_Q2rWWWvpL6xaMlpJyY90WBftag/export?format=csv&gid=625265890"
df_spells = pd.read_csv(spells_url)
df_spells.head()

import json
import hashlib

# %%
def row_to_spell(row):
  # Basic normalization
  components_str = row.get("Components", "")
  components_set = {comp.strip().upper() for comp in str(components_str).split(",")}
  spell_classes = [cls.strip() for cls in str(row.get("Class", "")).split(",") if cls.strip()]
  ability_checks = [cls.strip() for cls in str(row.get("Ability Check", "")).split(",") if cls.strip()]
  misc_tags = [cls.strip() for cls in str(row.get("Foundry Tag", "")).split(",") if cls.strip()]
  damages = [cls.strip() for cls in str(row.get("Damage Type", "")).split(",") if cls.strip()]
  saving_throws = [cls.strip() for cls in str(row.get("Saving Throw", "")).split(",") if cls.strip()]
  areas = [cls.strip() for cls in str(row.get("Area ABRV", "")).split(",") if cls.strip()]
  # Parse components if they are stored as a string like "V, S, M"
  components_str = row.get("Components ABVR", "")
  duration_type = row.get("Duration Type") if not pd.isnull(row.get("Duration Type")) else "timed"
  duration_unit = row.get("Duration Unit") if not pd.isnull(row.get("Duration Unit")) else "minutes"
  duration_amount = row.get("Duration Amount") if not pd.isnull(row.get("Duration Amount")) else 1
  range_distance = row.get("Range Distance") if not pd.isnull(row.get("Range Distance")) else "self"
  
  components_set = {comp.strip().upper() for comp in str(components_str).split(",")}
  spell = {
    "name": row.get("Spell Name", "Unnamed Spell"),
    "level": int(row["Level"][0]) if not pd.isnull(row.get("Level")[0]) else 0,
    "school": row.get("School ABRV", "E"),
    "time": [
      {
        "number": row.get("Casting Unit", 1),
        "unit": row.get("Casting Type", "Action"),
      }
    ],
    "range": {
      "type": row.get("Range Type", "Point"),
      "distance": {
        "type": range_distance,
        **({"amount": row.get("Range Unit")} if not pd.isnull(row.get("Range Unit")) else {})
      }
    },
    "duration": [
      {
        "type": duration_type,
        **({"duration": {
          "type": duration_unit,
          "amount": duration_amount,
          "upTo": True if row.get("Up To", "FALSE") == "TRUE" else False
        }} if duration_type == "timed" else {}),
        **({"concentration": True if row.get("Concentration", "FALSE") == "TRUE" else False} if duration_type == "timed" else {}),
      }
    ],
    "classes": {
      "fromClassList": [
        {
          "name": cls,
          "source": "VSTGCC"
        } for cls in spell_classes
      ]
    },
    "entries": [
      row.get("Description")
    ] + ([row.get("Clarification")] if not pd.isnull(row.get("Clarification")) else []) 
      + ([row.get("Table")] if not pd.isnull(row.get("Table")) else []),
    "source": "VestigiumConcordCitySpells",
    **({"entriesHigherLevel": [
        {
        "type": "entries",
        "name": "At Higher Levels",
        "entries": [row.get("Higher Levels", "")] if not pd.isnull(row.get("Higher Levels")) else []
        }
      ]} if not pd.isnull(row.get("Higher Levels")) else {}),
    "components": {
      "v": "V" in components_set,
      "s": "S" in components_set,
      "r": "R" in components_set,
      "t": "T" in components_set
    },
    **({"abilityCheck": ability_checks} if not pd.isnull(row.get("Ability Check")) else {}),
    **({"miscTags": misc_tags} if not pd.isnull(row.get("Foundry Tag")) else {}),
    **({"damageInflict": damages} if not pd.isnull(row.get("Damage Type")) else {}),
    "fluff": {
      "entries": [
      ] + ([row.get("Flavor")] if not pd.isnull(row.get("Flavor")) else [])
        + ([row.get("Alternative Flavor")] if not pd.isnull(row.get("Alternative Flavor")) else [])
        + ([row.get("Quotes")] if not pd.isnull(row.get("Quotes")) else []),
      "images": [
        {
          "type": "image",
          "href": {
            "type": "external",
            # "url": generate_icon("Spell", row.get("Spell Name", "Unamed Spell"))
          }
        }
      ]
    },
    **({"savingThrow": saving_throws } if not pd.isnull(row.get("Saving Throw")) else {}),
    **({"areaTags": areas} if not pd.isnull(row.get("Area ABRV")) else {}),
  }
  return spell

# %% [markdown]
# ## Convert All Spells and Export to JSON
#
# The following cell loops through each row in the DataFrame, converts it to the required JSON format,
# and then writes all spells into a JSON file wrapped in a compendium container.

# %%
spells_list = [
  row_to_spell(row)
  for index, row in df_spells.iterrows()
  if pd.notnull(row.get("Spell Name")) and str(row.get("Spell Name")).strip() != ""
]

# Items





In [42]:

items_url = "https://docs.google.com/spreadsheets/d/1I4FHncl40_xx1Udc_Q2rWWWvpL6xaMlpJyY90WBftag/export?format=csv&gid=876046336"
df_items = pd.read_csv(items_url)
df_items.head()

def row_to_item(row):
  properties = row.get("Property") if not pd.isnull(row.get("Property")) else []
  item = {
    "name": row.get("Name", "Generic Item"),
    "source": "VSTGCC",
    "type": row.get("Tye ABRV", ""),
    "rarity": row.get("Rarity", ""),
    "Value": row.get("Value", ""),
    "Weight": row.get("Weight", ""),
    "currencyConversion": "credit",
    **({"weaponCategory": row.get("Category", "") } if not pd.isnull(row.get("Category")) else {}),
    **({"properties": [f"VS{property[2:]}" for property in properties]} if not pd.isnull(row.get("Property")) else {}),
    **({"dmg1": row.get("Damage 1", "") } if not pd.isnull(row.get("Damage 1")) else {}),
    **({"dmg2": row.get("Damage 2", "") } if not pd.isnull(row.get("Damage 2")) else {}),
    **({"dmgType": row.get("Damage Type", "") } if not pd.isnull(row.get("Damage Type")) else {}),
    **({"range": row.get("Extracted Range", "") } if not pd.isnull(row.get("Extracted Range")) else {}),
    **({"entries": [
      row.get("Description", "")
    ], } if not pd.isnull(row.get("Description")) else {}),
    "images": [
      {
        "type": "image",
        "href": {
          "type": "external",
          # "url": generate_icon("Item", row.get("Name", "Generric Item"))
        }
      }
    ]
  }
  return item

items_list = [
  row_to_item(row)
  for index, row in df_items.iterrows()
  if pd.notnull(row.get("Name")) and str(row.get("Name")).strip() != ""
]

In [43]:
item_properties_url = "https://docs.google.com/spreadsheets/d/1I4FHncl40_xx1Udc_Q2rWWWvpL6xaMlpJyY90WBftag/export?format=csv&gid=1064461316"
df_item_properties = pd.read_csv(items_url)
df_item_properties.head()

def row_to_property(row):
  return {
    "abbreviation": f"VS{row.get("Name", "")[2:]}",
    "source": "VSTGCC",
    "page": 0,
    **({"entries": [row.get("Entry", "")] } if not pd.isnull(row.get("Entry")) else {}),
  }

item_property_list = [
  row_to_property(row)
  for index, row in df_item_properties.iterrows()
  if pd.notnull(row.get("Name")) and str(row.get("Name")).strip() != ""
]

# Classes

In [44]:
classes_url = "https://docs.google.com/spreadsheets/d/1I4FHncl40_xx1Udc_Q2rWWWvpL6xaMlpJyY90WBftag/export?format=csv&gid=1924660120"
class_tables_url = "https://docs.google.com/spreadsheets/d/1I4FHncl40_xx1Udc_Q2rWWWvpL6xaMlpJyY90WBftag/export?format=csv&gid=193036738"
df_classes = pd.read_csv(classes_url, header=1)
df_classes.head()

df_class_tables = pd.read_csv(class_tables_url, header=1)
df_class_tables.head()

def to_table(class_name: str):
  header = df_class_tables.iloc[1]
  def process(row):
    proficiency = row.get("Proficiency Bonus") if not pd.isnull(row.get("Proficiency Bonus")) else ""
    knownSpells = row.get("Spells Known") if not pd.isnull(row.get("Spells Known")) else ""
    maxSpellLevel = row.get("Max Spell Level") if not pd.isnull(row.get("Max Spell Level")) else ""
    Points = row.get("Points") if not pd.isnull(row.get("Points")) else ""
    spellSlots = row.get("Total spell slots") if not pd.isnull(row.get("Total spell slots")) else ""
    feature1 = row.get("Feature 1") if not pd.isnull(row.get("Feature 1")) else ""
    return [
        *([proficiency] if not pd.isnull(row.get("Proficiency Bonus")) else []),
        *([knownSpells] if not pd.isnull(row.get("Spells Known")) else []),
        *([maxSpellLevel] if not pd.isnull(row.get("Max Spell Level")) else []),
        *([Points] if not pd.isnull(row.get("Points")) else []),
        *([spellSlots] if not pd.isnull(row.get("Total spell slots")) else []),
        *([feature1] if not pd.isnull(row.get("Feature 1")) else [])
      ]
    
  labels = [
    *([f"{{@filter Proficiency Bonus|value|class={class_name}}}"] if not pd.isnull(header.get("Proficiency Bonus")) else []),
    *([f"{{@filter Spells Known|value|class={class_name}}}"] if not pd.isnull(header.get("Spells Known")) else []),
    *([f"{{@filter Max Spell Level|value|class={class_name}}}"] if not pd.isnull(header.get("Max Spell Level")) else []),
    *([f"{{@filter Points|value|class={class_name}}}"] if not pd.isnull(header.get("Points")) else []),
    *([f"{{@filter Total spell slots|value|class={class_name}}}"] if not pd.isnull(header.get("Total spell slots")) else []),
    *([f"{{@filter {header.get('Feature 1 Name')}|value|class={class_name}}}"] if not pd.isnull(header.get("Feature 1")) else [])
  ]
  tablish = [
    process(row)
    for index, row in df_class_tables.iterrows()
    if pd.notnull(row.get("Class")) and str(row.get("Class")).strip() == class_name
  ]

  return {
    "colLabels": labels,
    "rows": tablish
  }

def row_to_class(row):
  class_name = row.get("Name") if not pd.isnull(row.get("Name", "Generic Class")) else ""
  skills = row.get("Skills") if not pd.isnull(row.get("Skills")) else ""
  armors = row.get("Armor") if not pd.isnull(row.get("Armor")) else ""
  weapons = row.get("Weapons") if not pd.isnull(row.get("Weapons")) else ""
  saving_throw = row.get("Saving Throws") if not pd.isnull(row.get("Saving Throws")) else ""
  tools = row.get("Tools") if not pd.isnull(row.get("Tools")) else ""
  proficiency = row.get("Proficiency") if not pd.isnull(row.get("Proficiency")) else ""
  spellcasting_ability = row.get("Spellcasting Ability") if not pd.isnull(row.get("Spellcasting Ability")) else ""
  caster_progression = row.get("Caster Progression") if not pd.isnull(row.get("Caster Progression")) else ""
  
  classes = {
    "name": class_name,
    **({ "hd": {
      "faces": row.get("Hit Points ar 1st Level", ""),
      "number": 1
    }}),
    "proficiency": [
      prof[:3]
      for prof in proficiency
    ],
    "startingProficiencies": {
      "armor": [
        armor
        for armor in armors
      ],
      "weapons": [
        *[
          f"{{@item {weapon}|VSTGCC|{weapon}}}"
          for weapon in weapons
        ],
        "simple",
      ],
      "skills": [
        {
          "choose": {
            "from": [
              skill
              for skill in skills
            ],
            "count": 3
          }
        }
      ]
    },
    "startingEquipment": {
      "additionalFromBackground": True,
      "default": [
        "(a) a {@item rapier|phb}, (b) a {@item longsword|phb}, or (c) {@filter any simple weapon|items|source=phb|category=basic|type=simple weapon}",
        "(a) an {@item explorer's pack|phb} or (b) a {@item dungeoneer's pack|phb}",
        "{@item Leather armor|phb} and a {@item dagger|phb}"
      ],
      "defaultData": [
        {
          "a": [
            "rapier|phb"
          ],
          "b": [
            "longsword|phb"
          ],
          "c": [
            {
              "equipmentType": "weaponSimple",
              "quantity": 1
            }
          ]
        },
        {
          "a": [
            "explorer's pack|phb"
          ],
          "b": [
            "dungeoneer's pack|phb"
          ]
        },
        {
          "_": [
            "leather armor|phb",
            "dagger|phb"
          ]
        }
      ]
    },
    "spellcastingAbility": spellcasting_ability,
    "casterProgression": caster_progression,
    "cantripProgression": [
      row.get("0")
      for index, row in df_class_tables.iterrows()
      if pd.notnull(row.get("Class")) == class_name
    ],
    "classTableGroups": to_table(class_name)
  }
  return classes

classes_list = [
  row_to_class(row)
  for index, row in df_classes.iterrows()
  if pd.notnull(row.get("Name")) and str(row.get("Name")).strip() != ""
]



# Features

In [45]:
class_features_url = "https://docs.google.com/spreadsheets/d/1I4FHncl40_xx1Udc_Q2rWWWvpL6xaMlpJyY90WBftag/export?format=csv&gid=545140625"

def row_to_feature_entries(row):
  frame = pd.read_csv(class_features_url, header=1)
  frame.head()
  name = row.get("Name")
  classes = row.get("Class")
  parent = row.get("Parent")
  page = row.get("Page", 10)
  type = row.get("Type", "entries")
  entry = row.get("Entry")
  attribute = row.get("Attributes")
  
  return {
    **({"type": type} if not pd.isnull(row.get("Type")) else {"type": "entries"}),
    **({"name": name} if not pd.isnull(row.get("Name")) else {}),
    **({"page": page} if not pd.isnull(row.get("Page")) else {"page": 10}),
    "entries": [
       *([entry] if not pd.isnull(row.get("Entry")) else []),
       *[
         row_to_feature_entries(entry_row)
         for index, entry_row in df_class_features.iterrows()
         if pd.notnull(entry_row.get("Class")) and pd.notnull(entry_row.get("Parent")) and str(entry_row.get("Parent"))== name and str(entry_row.get("Class")) == classes
       ]
    ],
    **({"Attributes": attribute} if not pd.isnull(row.get("Attributes")) else {}),
  }

def row_to_features(row):
  frame = pd.read_csv(class_features_url, header=1)
  frame.head()
  name = row.get("Name")
  classes = row.get("Class")
  page = row.get("Page")
  subclass = row.get("Subclass")
  level = row.get("Level")
  type = row.get("Type")
  entry = row.get("Entry")

  return {
    **({"className": classes} if not pd.isnull(row.get("Class")) else {}),
    **({"name": name} if not pd.isnull(row.get("Name")) else {}),
    **({"subclass": subclass} if not pd.isnull(row.get("Subclass")) else {}),
    **({"page": page} if not pd.isnull(row.get("Page")) else {}),
    **({"level": level} if not pd.isnull(row.get("Level")) else {}),
    "source": "VSTGCC",
    "entries": [
      *([entry] if not pd.isnull(row.get("Entry")) else []),
      [
        row_to_feature_entries(entry_row)
        for index, entry_row in df_class_features.iterrows()
        if pd.notnull(entry_row.get("Class")) and pd.notnull(entry_row.get("Parent")) and str(entry_row.get("Parent")).strip() == name and str(entry_row.get("Class")).strip() == classes
      ]
    ]
  }

df_class_features = pd.read_csv(class_features_url, header=1)
df_class_features.head()

features_list = [
  row_to_features(row)
  for index, row in df_class_features.iterrows()
  if pd.notnull(row.get("Name")) and str(row.get("Name")).strip() != "" and not pd.notnull(row.get("Parent"))
]

# Generate whole file

In [46]:

def generate_date_last_modified_hash(doc) -> str:
  if isinstance(doc, dict):
    doc = dict(doc)
    doc.pop("_dateLastModifiedHash", None)

  json_str = json.dumps(doc, ensure_ascii=False, sort_keys=True)

  md5_hash = hashlib.md5(json_str.encode("utf-8")).hexdigest()
  return md5_hash

current_epoch = time.time()
compendium = {
  "_meta": {
    "sources": [
      {
        "json": "VestigiumConcordCitySpells",
        "abbreviation": "VSTGCC",
        "full": "Vestigium - Guide to Concord City",
        "url": "https://github.com/la-rockoteque/Vestigium/blob/main/out/Velum_Cineris;Vestigium_Guide_to_Concord_City.json",
        "authors": [
          "Velum Cineris"
        ],
        "convertedBy": [
          "Vincent Bernier"
        ],
        "version": "1.0"
      }
    ],
    "dateAdded": 1743879729,
    "dateLastModified": current_epoch,
    "_dateLastModifiedHash": "??????",
    "edition": "classic"
  },
  "currencyConversions": {
    "credit": [
      {
        "coin": "credit",
        "mult": 0.001,
        "isFallback": True
      }
    ]
  },
  "spell": spells_list,
  "item": items_list,
  "class": classes_list,
  "classFeatures": features_list,
  "itemProperty": item_property_list
}

new_hash = generate_date_last_modified_hash(compendium)
compendium["_meta"]["_dateLastModifiedHash"] = F"{new_hash}"
# Write the compendium to a JSON file
output_file = "out/Velum_Cineris;Vestigium_Guide_to_Concord_City.json"
with open(output_file, "w", encoding="utf-8") as f:
  json.dump(compendium, f, ensure_ascii=False, indent=4)

print(f"Export successful! The compendium has been saved as '{output_file}'.")

Export successful! The compendium has been saved as 'out/Velum_Cineris;Vestigium_Guide_to_Concord_City.json'.
